In [1]:
import os
os.chdir('/content/drive/MyDrive/workspace/Learning_Equality/paraphrase-multilingual-mpnet-base-v2-context')

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
!pip install -U transformers
!pip install sentencepiece
!pip install -U fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install datasets
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoConfig, AutoModel, AutoTokenizer, get_cosine_schedule_with_warmup
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold
from datasets import Dataset
import random
class CFG:
  ROW_DIR = Path('../data/row')
  PROCESSED_DIR = Path('../data/processed/train_data')
  TOKENIZER = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
  MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
  BATCH_PER_GPU = 128
  SEED=42
  NUM_EPOCHS=15
  LR = 1e-5
  NUM_GPUS=1 #4
  NUM_JOBS=1 #4
  AMP=True #True
  
def get_train_test_data(train, train_idx):
    train["fold"] = -1
    # 交差検証 用の番号を振ります。
    kf = KFold(n_splits=5, shuffle=True, random_state=CFG.SEED)
    for n, (train_index, val_index) in enumerate(kf.split(train_idx)):
        train.loc[train_idx[val_index], "fold"] = int(n)
    train["fold"] = train["fold"]
    return train

def get_path_list(df):
    topics_id2title = {k:v for k, v in zip(df.id.to_list(), df.title.to_list())}
    topics_id2description = {k:v for k, v in zip(df.id.to_list(), df.description.to_list())}
    topics_id2parents = {k:v for k, v in zip(df.id.to_list(), df.parent.to_list())}
    path_list = []
    for id in df.id.to_list():
        res_list = []
        while True:
            res_list.append(topics_id2title[id])
            id = topics_id2parents[id]
            if id=="":
                break
        path_list.append(" | ".join(res_list[::-1][:-1]))
    return path_list

In [7]:
topics_df = pd.read_csv(CFG.ROW_DIR / 'topics.csv')
content_df = pd.read_csv(CFG.ROW_DIR / 'content.csv')
sample_submission = pd.read_csv(CFG.ROW_DIR / 'sample_submission.csv')
correlations_df = pd.read_csv(CFG.ROW_DIR / 'correlations.csv')
content_df = content_df.fillna('')
topics_df = topics_df.fillna('')
# pathを取得する
topics_df['path'] = get_path_list(topics_df)
topics_df = topics_df[topics_df.has_content].reset_index(drop=True)
topics_df.drop(['description', 'channel', 'category', 'level', 'parent', 'has_content'], axis = 1, inplace = True)
content_df.drop(['description', 'kind',  'text', 'copyright_holder', 'license'], axis = 1, inplace = True)
train_idx = topics_df[~topics_df.id.isin(sample_submission.topic_id)].index
topics_df = get_train_test_data(topics_df, train_idx)
train_df = topics_df.join(correlations_df.content_ids.str.split(" ").explode())
train_df= train_df.reset_index(drop=True)
train_df = train_df.drop_duplicates(subset=['id', 'content_ids'])
train_df = train_df.rename(columns={'id':'topic_id', 'content_ids':'content_id'}).drop(columns = ['title', 'language'])
train_df = train_df[train_df.fold!=-1]
# tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL)
# tokenizer.add_tokens(["<|=t_sep=|>"], special_tokens=True)

In [8]:
topics_title_dict = {id:title for id, title in zip(topics_df.id, topics_df.title)}
topics_path_dict = {id:path for id, path in zip(topics_df.id, topics_df.path)}
content_title_dict = {id:title for id, title in zip(content_df.id, content_df.title)}

In [9]:
topic_input = [topics_title_dict[id]+"<|=t_sep=|>"+topics_path_dict[id] for id in train_df.topic_id]
content_title = [content_title_dict[id] for id in train_df.content_id]

In [10]:
topic_input[0]

'100 સુધીનો સરવાળો<|=t_sep=|>DIGITAL EDUCATION WITH MEHUL | ધોરણ ૩ | પ્રારંભિક ગણિત | 100 સુધીના સરવાળા અને બાદબાકી'

In [11]:
df = pd.DataFrame({"topic_input":topic_input, 
                  "content_title":content_title})

In [12]:
dataset = Dataset.from_pandas(df)

In [13]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses

In [14]:
train_examples = []
for t,c in zip(topic_input, content_title):
    train_examples.append(InputExample(texts=[t,c]))

In [15]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [16]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=128)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
# num_epochs = 10
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2187 [00:00<?, ?it/s]

In [ ]:
model.save("./paraphrase-multilingual-mpnet-base-v2-exp")